# State:  

{
input_h : [x,x,x,x,x]
server_load :[y,y,y] depends on input_h
}

# State+1:

# Goal: Train a policy that works for a wide range of wireless gain channels

In [1]:
import gymnasium as gym
from gymnasium.spaces import Box,Dict,Discrete
import numpy as np
import random
import ray
from ray.rllib.algorithms import ppo

class CustomEnv(gym.Env):
    def __init__(self):
        """
        Must define self.observation_space and self.action_space here
        """
        # Define action space: bounds, space type, shape

        # Bound: Action_space is based on the number of servers +1. For this case, we set it to 3. (3 servers)
        
        self.max_choices = 4
        self.count = 0
        #Space type: Better to use Box than Discrete, since Discrete will lead to too many output nodes in NN
        #Shape: rllib cannot handle scalar actions, so we turn it into a numpy array with shape (1,) 
        #self.action_space = Box(low = np.array([0]), high = np.array([self.max_choices]))
        self.action_space = Discrete(self.max_choices)
        # Bound: Observation_space
        # input_h is the channel gain between the access point and device x (), assume we have 5 devices
        # decisions is the choice made by the agent to decide the resource allocation of each device
        #server_load = load of each server at the moment
        #obs_low = np.zeros((self.obs_dim,))
        
        self.numServers = 3 # number of servers or base stations
        
        self.observation_space = Dict(
        {
         'input_h': Box(low = np.full(5, -np.inf), high = np.full(5, np.inf), shape = (5,), dtype = np.float),
         #'decisions': spaces.Box(low =np.full(5, 0), high = np.full(5,4), shape = (5,), dtype = np.intc)
         'server_load': Box(low = np.zeros(self.numServers), high = np.full(3, np.inf), shape = (3,), dtype = np.float)
        })        
        
        self.current_obs = None
        self.log = ''

    
    def reset(self, next_input):
        """ 
        Returns: observation of the initial state
        Reset environment to initial state so that a new episode independent of previous ones can start
        We want to reset input_h to next set of input_h, rest to 0.
        """
        input_h = next_input
        server_load = np.zeros(self.numServers)
        #decisions = np.full(5,0)
        self.current_obs = {
            'input_h': input_h,
            'server_load' : server_load
            #'decisions': decisions
        }
        self.count = 0
        
        return self.current_obs

    def step(self, action):
        """
        Returns the next observation, reward, done and optinally additional info
        """
        #Action looks like np.array. convert to float for easier calculation.
        print(self.count)
        choice = random.choice(action)
        print(f'Chosen choice: {choice}')
        self.log += f'Chosen action: {choice}\n'
        
        #Compute next observation
        self.current_obs['server_load'][choice] += 1
        next_obs = self.current_obs
        
        #Compute reward
        if self.count < 4:
            reward = 0
        if self.count ==4:
            reward = self.current_obs['server_load'][0]*self.current_obs['input_h'][self.count] + self.current_obs['server_load'][1]*self.current_obs['input_h'][self.count] + self.current_obs['server_load'][2]* self.current_obs['input_h'][self.count]
            print("reward",reward)
        done = False
        if self.count == 4:
            done = True
        self.count+=1
        self.current_obs = next_obs
        return self.current_obs, reward, done, {}
    def render(self):
        """
        Show current environment state
        Must be implemented, if not important, can have an empty implementation
        """
        pass

    def close(self): # optional
        """
        Used to clean up all resources. Optional
        """
        pass

    def seed(self): # optional
        """
        Used to set seeds for environment's RNG for obtaining deterministic behavior. Optional
        """
        return

Instructions for updating:
experimental_relax_shapes is deprecated, use reduce_retracing instead


In [2]:
env = CustomEnv()

for _ in range(10):    
    arr = np.random.rand(5)
    print(arr)
    obs = env.reset(arr)
    while True:
        print("test")
        action = env.action_space.sample()
        
        obs,r,done, _ = env.step(action)
        #assert np.all(obs <= env.observation_space.high), f"Observation {obs} does not respect observation space upper bound"
        if done:
            break

C:\Users\donvi\.conda\envs\DROO\lib\site-packages\ipykernel_launcher.py:33: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
C:\Users\donvi\.conda\envs\DROO\lib\site-packages\ipykernel_launcher.py:35: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


[0.39290819 0.38042677 0.19196273 0.82057973 0.33301518]
test
0


TypeError: object of type 'int' has no len()

In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [12]:
states = env.observation_space.shape
actions = env.action_space.n

In [13]:
actions

4